<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Chemicals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chemicals**

**Authors: Bridget Childs, Ananya Gangadhar, Jacqueline Wong**

[Monroe]: # (I'm realizing that you hadn't found all of my comments, so I've moved this out of the comments into the main text to ensure you find it. Make sure to open text boxes including this one! and the ones below to see if there are any comments!)

[Monroe]: # (The progress since the previous submission seems to be minimal. The analysis that you have done is both relatively simple and appears to have significant errors. Pick up the pace, review each other's work, bring healthy skepticism to your design, make your default position be that you DON'T believe what your team mates have done and make them convinced you. Design is so important because it is when we create the dream of how we are going to change the real world. You want to make sure that you are creating design recommendations that will work. Design isn't for a grade. Design mistakes are expensive at best and deadly at worst. That is why engineering skepticism is so important.)

[Monroe]: # (This was my feedback from the previous submission. You are making reasonable progress. Fix your estimates of flow rates and volumes, design the plumbing system, specify the pumps required, review your report for spelling errors [too bad that spell checkers don't work yet], design the storage tanks, etc.)



In [9]:
!pip install aguaclara
!pip install molmass

In [10]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-05-16 16:22:27--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3052 (3.0K) [text/plain]
Saving to: ‘Fairmont.py.1’

Fairmont.py.1       100%[===================>]   2.98K  --.-KB/s    in 0s      

2020-05-16 16:22:27 (51.9 MB/s) - ‘Fairmont.py.1’ saved [3052/3052]



## **Task Definition**

![chemicals task definition](https://drive.google.com/uc?id=1IujmG6g8i9THTNARiC67swyN3FfKiPDl)

The Chemicals team will be in charge of designing the chemical dosage system. This will include:
- Determining coagulant and polymer dosage
- Designing chemical storage, feed lines, and flow control system.
- Calculating feed hydraulics and plumbing
- Identifying available vendors.

Additionally, our storage system design will include the number of storage tanks, tank size, location, footprint, and safety add-ons in order to meet OSHA requirements (e.g. spillover capture, eye washes, safety showers). Information will be primarily sourced from government regulations, the Fairmont Sedimentation Plant Scope of Work document, relevant vendors, and similar go-by systems.  

## **Executive Summary**

(Highlight dominant constraints and your recommendation for the design.)

## **Constraints**

* Design flow of the plant is 720 cfs.
* Effluent arsenic levels should be under 10 ug/L.
* Effluent TOC levels should be under 3 mg/L.
* Effluent turbidity should be under 3 NTU.
* Chemicals will be stored in an open tank farm and have at least a 30 day capacity  (Scope of work, pg 18).
* Proposed tank farm should have area under 200'x100'
* All hazardous chemicals must be stored in an area that is designed to contain spills ([OSHA regulations](https://www.safetyinfo.com/safe-chemical-storage-osha-requirements-free-index/)).
* All design values should be justifiable using design equations.
* The chemical feed system should be easy to understand and operate, and should have relatively low maintenance.
* The designed system should have a feedback loop wherein the chemical dosage is adjusted accordingly with a change in the plant flow rate.

**CEE 4590 Logistical constraints**
* Information in Scope of Work is often missing citations, and we are unsure of the source and degree of certainty of parameters like expected FeCl3 dosage.
* At present, we have no communication with the LA engineers on this project, only the Stantec engineers, Mickey and Tori, who are working on Stantec's own proposal for this project. 
* It will be challenging to procure cost estimates from vendors since they are typically reluctant to share such information with students.
* It may be wise to add more redundancy to the chemical storage system than the recommended 30 days for situations like a shutdown due to a pandemic when supply chains are disrupted.. 

## **Physics**

Since the Chemicals Team task concerns mainly the storage and delivery of coagulant and polymers, it is important to consider the energy and space that these chemicals will occupy. The **Energy equation** accounts the energy losses in fluid mechanics. Below is the general form, which will be useful as it is tailored to the chemical dosage design. [(AguaClara Textbook)](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-review-review-fluid-mechanics-4)

$$\frac{p_{1}}{\rho g} + z_{1} + \alpha_{1} \frac{\bar v_{1}^2}{2g} + h_P = \frac{p_{2}}{\rho g} + z_{2} + {\alpha_{2}} \frac{\bar v_{2}^2}{2g} + h_T + h_L$$

Where:

p = pressure

ρ = density 

z = height 

v = velocity 

hp, ht = viscueous frictional loss due to pumps, turbines

hl = head loss


Head loss comprises of major losses from friction and minor losses from flow expansions, elbows. Major and minor loss equations are discussed below. 

 $$h_L = h_{\rm{f}} + h_e$$

Minor losses may be summarized as the equation below. Minor losses occur at flow expansions, elbows, and valves. K will vary depending on the kind of minor loss. 

$$h_e = \sum K \frac{\bar v_{out}^2}{2g} =\left( \frac{A_{out}}{A_{in}} -1 \right)^2 \frac{\bar v_{out}^2}{2g}$$

The **Darcy-Weisbach equation** solves for major losses in the head loss equation. Major losses comprise the frictional losses that occur in pipes. For this equation, it will be important to know the length and diameter of the pipes in our delivery system. Equation found in the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-darcy-weisbach). Additionally, there is an accompanying python function in the AguaClara python package to calculate major losses.

$$h_{\rm{f}} \, = \, {\rm{f}} \frac{L}{D} \frac{\bar v^2}{2g}$$

$$h_{\rm{f}} \, = \,{\rm{f}} \frac{8}{g \pi^2} \frac{LQ^2}{D^5}$$

Such that: 

hf = major loss 

f = Darcy friction factor
 
L = pipe length

Q = pipe flow rate
 
D = pipe diameter

AC Python Function = pc.headloss_fric(FlowRate, Diam, Length, Nu, PipeRough)


Other types of energy that may be of use to our chemical dosage system will be kinetic and potential energy, listed below. These are more general, but may be important to consider, especially with respect to the site's geography and elevation.   

$$ E_{potential} = mgh \,\,\,\,\, E_{kinetic} = mv^2/2$$

Ultimately, whatever power is neeeded to run the chemical dosage system will be supplied by pumps. Knowing the amount of power required will help choose the appropriate pump for the design. Below is the equation for power. E represents energy and t represents time. 

$$ P = \frac{E}{t} $$

Lastly, mass conservation will important to consider during our calculations. This will help us better understand how much storage will be required when considering the required dosage. 

$$Q_{in}C_{in} = Q_{out}C_{out}$$

## **Table of potential strategies to meet the constraints and treatment objectives**

| strategy | advantages  | disadvantages | data needed |
|---|---|---|---|
| open tank farm vs. enclosed storage | go-by, less expensive | less protection | weathering risk |
| 3 storage tanks vs. using even more | less complexity | lower safety factor | storage area |
| choosing vendor before choosing tank size | easier to pick model | more restricted | vendor details |
| 30-day storage vs. keeping even more | resilience (e.g. COVID-19) | more expensive | area, budget |
| single vs. multiple dosage system per train | maintenance, operator ease | fewer options if pipe fails | clogging risk |



## **Vendors and contact information file**

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [11]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design


# **Shared Assumptions**

The Chemical team will be utilizing the [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) file in conjunction with the other design teams in order to share assumptions from the Fairmont Sedimentation Tank Scope of Work document. It will also create dependency between the chemical design below and the designs of other teams.  

# **Major Decisions**

**Ferric Chloride Coagulant**

The Fairmont Sedimentation Plant Scope of Work Document notes that two types of products will be used: a coagulant (specifically, ferric chloride) and emulsion polymer. Due to communication constraints with engineers at the Los Angeles Department of Water and Power, we are unlikely to fully understand the rationale behind the selection of ferric chloride as the coagulant. However, consider that it was already specified, we have decided to move forward with ferric chloride as the coagulant of choice.

Since the Scope of Work only states the average dose of ferric chloride is 13.14 mg/L. We have conservatively assumed that the maximum dose of ferric chloride will be 30 mg/L. This is slightly more than twice the average dose. 

**Emulsion Polymer**

On the other hand, the type of emulsion polymer was not specified in the Scope of Work document. There are many types and blends of polymers that are available in the water treatment market, many of which are proprietary and are <del>vague</del>  in chemical makeup. 

Due to limitations of this design project, we will not be able to select an emulsion polymer with confidence without bench testing capabilities. However, we do know that the polymer is going to be cationic, and there are multiple vendors available. We will research a few options from different vendors and select a representative emulsion polymer to provide an estimate for density, and viscosity. More information about the representative polymer will be provided in the design section. 

**Other Chemicals**

It was noted by other teams that a high coagulant dosage could significantly alter the pH of the water. Since the Scope of Work did not specify the need for other chemical products, we made the assumption that the acid neutralizing capacity of raw water entering the Fairmont Sedimentatino Plant is adequate, and that additional chemicals beyond the coagulant and emulsion polymer are not required.

**Chemical Storage**

As both the Cottonwood Treatment Plant and Los Angeles Aqueduct Filtration Plant use ferric chloride, we initially thought of estimating tank volume using measurement tools from Google Earth. However, this proved to be quite a challenge and perhaps not the most accurate approach since the other plants may have different treatment goals.

Instead, we decided to work backwards from the annual masses of ferric chloride and emulsion polymer used specified by the Scope of Work to calculate the required tank volumes, assuming that they will be stored at the highest concentration commercially available.

**Pump Selection**

The Pennsylvania Department of Environmental Protection's water treatment training outlines popular chemical dosing pumps. It recommends peristaltic (tube) pumps for feed rates less than .1 gallon per hour (gph), Solenoid metering pumps for feed rates between .1 and 10 gph, and positive displacement pumps for feed rates less than 600 gallons per hour. The maximum feed rate will help decide what pump will be appropriate. 

For our design, we have selected a positive displacement diaphragm pump. The diaphgram pump uses a flexible membrane and valves to push water through the pump chamber. The membrane stretches outward to let water into the chamber and stretches inward to displace and push the water outward. The oscillating motion of the membrane may also be achieved with a piston or plunger. 

![Positive Displacement Diaphragm Pump](https://www.dienerprecisionpumps.com/files/content%20bilder/info/190726/190726-diaphragm-pump.jpg)

Figure provided by [Diener Precision Pumps](https://www.dienerprecisionpumps.com/en/positive-displacement-pumps.html).

**Accessory Equipment** 

The Pennsylvania Department of Environmental Protection also recommends the following accessory equipment for a liquid feed system: 

1. Calibration Chamber – to measure output of pump.
2. Pressure Relief Valve – to protect piping from high pressures and ensure isobaric conditions.
3. Backpressure Valve – to maintain a steady back pressure for chemical feed system.
4. Anti-siphon Valve – to prevent back flow into the chemical feed system.
5. Polymer Blending Unit - to active and mix the polymer before distribution to the treatment train. 
6. Spill Containment Zone - area under the storage of chemicals to ensure spillage will not spread. Requires grating, leak detection probe, and chemical resistent coating. 

The go-by provided by this guide will be helpful for us to include and account for the necessary hydraulic valves in a typical chemical feed system. Additionally, these additions will likely be inexpensive and will not contribute majorly to the plan view area of the design. For safety, we will also provide the following to meet OSHA requirements:

7. Eye Washes
8. Safety Showers

# **Next Steps and Challenges**

1. Sketch out chemical dosing system design: 
- Select tank farm location
- Select number of tanks, pumps
- Design pipe plumbing for dosage system
2. Update the vendors table with relevant vendors. 

# **Design**

[/Monroe/]: # (Begin by writing several paragraphs with equations explaining your design process. Then do the python analysis.)

[Chemicals]: # (Just found these comments yesterday, but definitely working on this!)

[/Monroe/]: # (The tank volume calculation is incorrect. The density_fecl3 is NOT the concentration of the FeCl_3. You have to figure out what the 40% solution means and then turn that into the concentration of the FeCl3 stock solution.) 

[Chemicals]: # (Corrected by Ananya in the design code)

[/Monroe/]: # (Your design process for the pipe size and pump power is not robust. The goal is to develop design processes that work reliably even if the basis of design changes. The pipe diameter for the chemical feed system needs to change automatically if the design chemical dose changes or if the type of coagulant changes. Use the AguaClara equations that calculate pipe diameters.) 

[/Jillian/]: # (Notes from 5/5 Class: There is pipes.py under AC. Some have "this is minimum blahblah you need". Min pipe diam function? SDR-26 is standard? Will chug out an actual pipe diameter... There are other functions too. Recall: Talk about the nominal diameter. But this is not the same as inside/outside diameter! Just make sure to be explicit about what diameter we are using in the design.)

[Chemicals]: # (This is our next big task to work on!)

[/Monroe/]: # (Start thinking about how many coagulant feed systems will be needed based on the number of rapid mix units. Pull that information from fmt. see my example below.)

[Chemicals]: # (As mentioned in preliminary presentations, we need more than 1. We will likely go with 2 per unit to keep the hydraulics simple, but also to account for redundancy.)

[/Monroe/]: # (You don't seem to be accounting for the fact that there are multiple chemical injection points and that each one will require a chemical dosing sytsem. Use the number of rapid mix units in the fairmont.py. You can access those values using fmt. notation. If this doesn't make sense please ask me or a TA)

[/Jillian/]: # (Consider something similar to P&ID diagram in sketch. Say not to scale. Okay to keep 3D, make clear tank and containment zone. Make clear method of thinking, mention areas that we are struggling in?)

[Chemicals]: # (Also, a task for the upcoming week is to start sketching designs. )

### **Maximum coagulant dosage in plant**

The first step in the design process is to determine the maximum coagulant (FeCl3) dosage expected in our system, given the design flow rate in the plant, calculated by conservation of mass: 

$$Q_{max,FeCl3} = \frac{Q_{des}C_{max,FeCl3}}{C_{stock,FeCl3}}$$
where

Q<sub>des</sub> = design flow rate of the plant = 720 cfs

C<sub>max,FeCl3</sub> = maximum coagulant dose used in the plant = 30 mg/L (assumed maximum, at least twice the average dose from Scope of Work)

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

[/Monroe/]: # (How do you know that this interpretation is correct? When I google this I get many interpretations of what a percent solution means and given the densities involved it is critical that you use the correct definition. I suggest checking with a manufacturer and include a reference to the definition of percent. This is another example of engineers using units that are unhelpful! Below you sometimes assume that the density is 1.4 g per mL and sometimes you assume the concentration is 0.4 g per mL. Those assumptions are incompatible. Figure this out!)

The necessary unit conversions have been left out of these equations as those can by easily calculated in Python using the pint library.

[/Monroe/]: # (Thank you for leaving the unit conversions out of the equations.)

In [0]:
#relevant parameters related to ferric chloride: 
fecl3_strength = 0.4 # Active strength of ferric chloride solution
density_fecl3 = 1.356*u.g/(u.mL) # for 40% m/v FeCl3, Brenntech Connect 
mu_fecl3 = 0.12 * u.N * u.s / u.m**2 # include this in the text above and cite your source for this estimate.
nu_fecl3 = mu_fecl3/density_fecl3

In [13]:
# Calculating the maximum coagulant dose in the plant from SOW
fecl3_dose_max = 30 * u.mg/u.L 
# add this to our Basis of design in fmt. 
# Done, but I keep getting the error 'AttributeError: module 'Fairmont' has no attribute 'Fe_c_max', so we'll change code later]
fecl3_stock = 400 * u.g/u.L # conc of coagulant stock in tank
Q_fecl3_max_dose = (fmt.q_max * fecl3_dose_max/fecl3_stock).to(u.mL/u.s)
print('The total maximum ferric chloride dosing flow rate is',Q_fecl3_max_dose.to(u.gal/u.hour), "(",Q_fecl3_max_dose.to(u.mL/u.s),")")

The total maximum ferric chloride dosing flow rate is 1454 gallon / hour ( 1529 milliliter / second )


In [14]:
# Calculating max coagulant dose in the plant from Hering & Kneebone 2002 (p.169)
fecl3_dose_avg_CTP = 5.7 * u.mg/u.L # Hering & Kneebone 2002
fecl3_dose_max = fecl3_dose_avg_CTP * 2 # Assume that maximum dose is double

fecl3_mass_delivery_rate = (fmt.q_max * fecl3_dose_max)/fecl3_strength
fecl3_vol_delivery_rate = fecl3_mass_delivery_rate/density_fecl3
print("The total maximum feed rate of ferric chloride solution is", fecl3_vol_delivery_rate.to(u.gal/u.hour), "(",fecl3_vol_delivery_rate.to(u.mL/u.s),")")

The total maximum feed rate of ferric chloride solution is 407.5 gallon / hour ( 428.5 milliliter / second )


[/Monroe/]: # (Fix the incorrect assumptions for the stock concentration above.)

[/Monroe/]: # (Provide a basis for your conclusion about what type of pump to use. Note that the flow rate calculation is incorrect for multiple reasons: wrong stock concentration, wrong number of rapid mix units)

Pump selection will be made based on the coagulant maximum flow rate required for each pump. This will have to do with the total maximum feed rate of coagulant provided above. It will also depend on the the number of pumps selected based on the storage tank design. The threshold for a solenoid pump is 10 gph, so this design will likely utilize the positive displacement diaphragm pumps, whose threshold are 600 gph. 

### **Storage volume of coagulant for 30 days**

[/Monroe/]: # (I believe the SOW provides an estimate of the annual use of FeCl3. I'd suggest using 1/12 of that estimate to get the storage requirement. Note that your first appoach is going around in circles because we estimated by the average coagulant dose based on the annual usage as reported in the SOW. So eliminate the first method.)

The next step is to determine the storage volume for 30 days worth of coagulant needed to run the plant. We assume that the plant is running at average conditions for this duration. The method to calculate the volume of coagulant needed over 30 days is to use the annual mass estimate of coagulant given in the scope of work, using the equation

$$V_{t,FeCl3} = \frac{M_{ann, FeCl3}}{C_{stock, FeCl3}} $$
where

M<sub>ann,FeCl3</sub> = annual estimate of mass of FeCl3 needed = 4142 tons/yr (from scope of work)

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

The SOW estimate will be used to calculate the storage since it yields the larger total feed rate required for the coagulant, ferric chloride. 

In [16]:
#volume FeCl3 for 30 days via annual mass estimate
fecl3_ann_est = 4142 * u.tons
fecl3_month_mass = fecl3_ann_est *u.month/u.year 
fecl3_month_vol = fecl3_month /FeCl3_stock
print("The volume of FeCl3 needed for 30 days based on the SOW annual estimate is", fecl3_month_vol.to(u.kgal), "(",fecl3_month_vol.to(u.ML),")")

NameError: ignored

[/Monroe/]: # (Include a discussion after your calculations explaining the significance of the results.)
  
To get a picture of how many tanks at the LAAFP would be required to store this volume, Google Earth was utilized to observe the existing LAAFP tank farm. Three types of tanks were identified. Below are calculations how many of each type of tank would be required to store the calculated volume. These estimates will help guide us when deciding on the number of tanks to choose.

![height of tank types 1, 2](https://drive.google.com/uc?id=1ueHQIRGAMqFvfSu6N_S7mmcFO7Y4VeaX)

![diameters of tank types 1, 2](https://drive.google.com/uc?id=1pSN_n2bjSVUst1jzg4zLsPVGoJF-KeUS)

In [0]:
#LAAFP tank volume for type 1 (larger white tanks on the left)
LAAFP_tank_1_height = 7.5 * u.m
LAAFP_tank_1_diam = 6 * u.m
LAAFP_tank_1_volume = np.pi * (LAAFP_tank_1_diam/2)**2 * LAAFP_tank_1_height
LAAFP_tank_1_number = np.ceil(fecl3_month_vol.to(u.L)/LAAFP_tank_1_volume.to(u.L))
print("The volume estimate for LAAFP tank type 1 is",LAAFP_tank_1_volume.to(u.kilogallon), "(",LAAFP_tank_1_volume.to(u.ML),")")
# Monroe stripped out the "dimensionless" unit to make the printouts clearer.
# Thanks!
print((LAAFP_tank_1_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 1 to store", fecl3_month_vol.to(u.kL), "(",fecl3_month_vol.to(u.ML),")")

#LAAFP tank volume for type 2 (smaller white tanks on the right)
LAAFP_tank_2_height = 7.5 * u.m
LAAFP_tank_2_diam = 5 * u.m
LAAFP_tank_2_volume = np.pi * (LAAFP_tank_2_diam/2)**2 * LAAFP_tank_2_height
LAAFP_tank_2_total = LAAFP_tank_2_volume * 3
LAAFP_tank_2_number = np.ceil(fecl3_month_vol.to(u.L)/LAAFP_tank_2_volume.to(u.L))
print("The volume estimate for LAAFP tank type 2 is", LAAFP_tank_2_volume.to(u.kilogallon),"(",LAAFP_tank_2_volume.to(u.ML),")")
print((LAAFP_tank_2_number.to(u.dimensionless)).magnitude,"tanks are required for tank type 2 to store", fecl3_month_vol.to(u.kL),  "(",fecl3_month_vol.to(u.ML),")")

![height and diameter of tank type 3](https://drive.google.com/uc?id=1Op3aap9rhu6s8SCLtirSz8PZfWrZLp8x)

In [0]:
#LAAFP tank volume for type 3 (blue-green tanks)
LAAFP_tank_3_height = 11 * u.m
LAAFP_tank_3_diam = 6 * u.m
LAAFP_tank_3_volume = np.pi * (LAAFP_tank_3_diam/2)**2 * LAAFP_tank_3_height
LAAFP_tank_3_total = LAAFP_tank_3_volume * 2
LAAFP_tank_3_number = np.ceil(fecl3_month_vol.to(u.L)/LAAFP_tank_3_volume.to(u.L))
print("The volume estimate for LAAFP tank type 3 is", LAAFP_tank_3_volume.to(u.kilogallon),"(",LAAFP_tank_3_volume.to(u.ML),")")
print((LAAFP_tank_3_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 3 to store", fecl3_month_vol.to(u.kL),  "(",fecl3_month_vol.to(u.ML),")")

Considering that there are 2-3 tanks for each tank type in the Los Angeles Aqueduct Filtration Plant, the above estimates of having 3-6 tanks seem relatively reasonable. These numbers also meet the minimum 3 tanks that we would like to have for redundancy – one for use, one for back-up, and one for additional back-up during maintenance. We have decided to move forward with taller tanks similar to the blue-green type 3 tanks to save space.

### **Annual cost of coagulant**

[/Monroe/]: # (I don't have any confidence that the equation below is correct. The random ratio can't be right. And I don't know where you got the cost or what the units on the cost are. Is this cost per lb of FeCl3 or is it cost per lb of solution?)

[Research](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144) into vendors showed that for bulk orders, the cost of 40% FeCl3 solution is around $0.2243/lbs. This value can be used to calculate the annual cost of coagulant using the formula

$$Annual\: cost\: of\:FeCl3 = (unit\: cost\:of\:FeCl3)*V_{30,FeCl3}* \rho_{FeCl3}$$

where 

V<sub>30,FeCl3</sub> = Volume needed to store 30 days worth of 40% FeCl3 solution (calculated above)

$\rho$<sub>FeCl3</sub> = density of 40% FeCl3 solution = 1.356 g/mL ( [BrennTag Connect](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144))

[/Monroe/]: # (Here you give a difference density of the stock solution. Getting these numbers right is a first step in your design. Make sure to fix this asap.)

[/Monroe/]: # (I believe the SOW include a cost for the coagulant. Does your calculation agree?)

In [0]:
#cost of FeCl3 
unit_cost_fecl3 = 0.2243 * u.USD/u.lbs # from Brenntag vendor
ann_cost_fecl3 = unit_cost_fecl3*fecl3_month_mass* 1*u.year / time
print('The annual cost of FeCl3 is', ann_cost_fecl3.to(u.USD))

SOW_ann_cost_fecl3 = 2100000 *u.USD
cost_ratio = ann_cost_fecl3/SOW_ann_cost_fecl3
print("The cost_ratio based on vendors is", cost_ratio.to(u.dimensionless).magnitude, "the cost based on the scope of work")

The answer calculated above is roughly 3 times the amount estimated in the scope of work. Usually, budget predictions are conservative which makes the ratio result surprising. Since we do not have a better idea of where the SOW price estimate comes from, we will leave our calculations as is. It is possible that LA engineers have a better unit pricing than the price posted online. 

### **Storage Volume & Max. Feed Rate of Emulsion Polymer**

Calculating the storage volume and maximum feed rate of the emulsion polymer is done similarly to the calculations used for ferric chloride. The storage volume and maximum feed rate of the emulsion polymer may be done either by using the annual polymer mass estimate from the SOW or the average dose of polymer used at the SOW. The two methods are yielding two drastically different results. The Chemicals team still needs to decide which result is more reasonable. 

The Tramfloc 300-399 Series Cationic Emulsion Polymer was selected for this design as a representative polymer. The polymer's safety data sheet may be found [here](http://tramfloc.com/material-safety-data-sheets/).

In [0]:
#Tramfloc 300-399 Series Cationic Emulsion Polymers properties: 
density_polymer = 1.01*u.g/(u.mL) # for 35% active content polymer
polymer_strength = 0.35 # Active strength of polymer solution
polymer_stock = 5 *u.g/u.L 
mu_polymer = 1 * u.N * u.s / u.m**2  #dynamic viscosity = 1000 cps 
nu_polymer = mu_fecl3/density_fecl3

In [0]:
# Based on annual estimate provided in the SOW
polymer_ann_mass = 165 * u.tons/u.year
polymer_ann_vol = polymer_ann_mass/density_polymer
print("The maximum polymer feed rate based on the SOW annual estimate is",polymer_ann_vol.to(u.gal/u.hour),"(",polymer_ann_vol.to(u.ml/u.sec),")")

In [0]:
# Based on the dosing rate at the Cottonwood plant (Hering & Kneebone 2002 (p.169))
polymer_dose_avg_CTP = 1.7 * u.mg/u.L
# Assume that maximum dose is double
polymer_dose_max = polymer_dose_avg_CTP * 2

polymer_mass_delivery_rate = (fmt.q_max * polymer_dose_max)/polymer_strength
polymer_vol_delivery_rate = polymer_mass_delivery_rate/density_polymer
print("The maximum feed rate of polymer solution is", polymer_vol_delivery_rate.to(u.gal/u.hour),"(",polymer_vol_delivery_rate.to(u.mL/u.s),")" )

In [0]:
#volume of polymer for 30 days via annual mass estimate (SOW)
polymer_mass_month = polymer_ann_est *u.month/u.year 
polymer_vol_month = polymer_mass_month/density_polymer
print("The mass of polymer needed for 30 days based on the SOW annual estimate is", polymer_mass_month.to(u.kilopound), "(",polymer_mass_month.to(u.Mg),")" )
print("The volume of polymer needed for 30 days based on the SOW annual estimate is", polymer_vol_month.to(u.kilogallon),"(",polymer_vol_month.to(u.kL),")")

In [0]:
#volume FeCl3 for 30 days via dosage at the Cottonwood Plant
time = 30 * u.days
tank_volume_polymer = polymer_dose_avg_CTP/polymer_stock* time * fmt.q_ave
polymer_month_mass = tank_volume_polymer*density_polymer
print("The mass of 35% polymer solution needed for 30 days based on the recommended dosage is", polymer_month_mass.to(u.klbs), "(",polymer_month_mass.to(u.Mg),")" )
print ("The volume of polymer needed for 30 days based on the recommended dosage is",tank_volume_polymer.to(u.kgal), "(",tank_volume_polymer.to(u.kL),")" )

### **Pipe Design**

[Chemicals]: # (TBC: Need to connect this to the pump selection process as well as incorporate this into the pipe diameter selection from AC Python Package)

[Monroe]: # (Okay, you missed this last time. If you have questions about how to do these calculations, bring me in to a meeting!)

[Monroe]: # (Design for the maximum chemical flow rate. It doesn't make sense to design a chemical feed system that fails when the flow rate is greater than the average flow. Don't forget to include the number of chemical dosing locations.)

The design below calculates the optimal nominal diameter for a schedule 40 pipe. First, the function ac.diam_pipe was used to find the optimal inner diameter. This is then used to find the corresponding schedule 40 inner diameter and nominal diameter. The inputs the function considered where: the pipe flow, the acceptable head loss, the pipe length, the coagulant viscosity, the roughness of the pipe, and the minor losses. 

The design will include 2 feed lines per rapid mix unit for redundancy purposes. It is also assumed that 1 m of head loss is acceptable to start. 

In [0]:
num_feed_pipes = 2*fmt.n_floc_mech #two pipes for redundancy
q_treatment_train = fmt.q_max/ (fmt.n_floc_mech-1)
feed_pipe_q = q_treatment_train * fecl3_dose_max / fecl3_stock 
print("The maximum flow rate is per feed is",feed_pipe_q.to(u.gal/u.hour), "(",feed_pipe_q.to(u.ml/u.s),")")
head_loss = 1 *u.m #1 m headloss to start
diam_pipe = ac.diam_pipe(feed_pipe_q, head_loss, pipe_l, nu_fecl3, e, num_elbow*k_elbow) #inner diameter
print("The optimal pipe diameter for the design is",diam_pipe.to(u.inch))

av_pipe_array = ac.ID_sch40(ac.ND_all_available()) #returns all ID for sch40 
ID_sch40 = ac.ceil_nearest(diam_pipe, av_pipe_array)

#use index of ID to find value of ND 
index = np.where(ID_sch40 == av_pipe_array) #index of ND and ID
ND_array = ac.ND_all_available()
ND_sch40 = ND_array[index][0]
print("The nominal diameter of the schedule 40 pipe is",ND_sch40)

actual_hl = ac.headloss_pipe(feed_pipe_q, ID_sch40, pipe_l, nu_fecl3, e, num_elbow*k_elbow)
print("The actual headloss of the current pipe design is", actual_hl) #if you're willing to have this amount of hl, design is okay. 

#pressure = rho * g * h, use density of stock solution bc significant diff in density 

In [0]:
#rough estimates of parameters
elevation_tank = 0 *u.feet
elevation_rapid_mix = 10 *u.feet #max. elevation change based on SOW elevation map 
pipe_l = 1000 *u.feet 
num_elbow = 4
k_elbow = .3 #Frank Fluid Mechanics pg 391, assumed 90 deg. regular flanged 4 in. nominal diameter
temp = 20 *u.degC
e = .0015 *u.mm #Frank Fluid Mechanics pg 371, assumed plastic

#calculating power pump 
head_pump = actual_hl + elevation_rapid_mix - elevation_tank
power_pump = flow_coagulant*density_fecl3*ac.GRAVITY*head_pump

print("The power required to pump the coagulant to the treatment train is",power_pump.to(u.watt))

### **Accessory & Safety Equipment**

Add pictures 


### **Plan View Area**

As the open tank farm will store ferric chloride coagulant for the rapid mix unit process as well as emulsion polymer for residuals handling, we suggest swapping the initial proposed open tank farm and truck staging areas from the layouts suggests in the Fairmont Sedimentation Plant Scope of Work document. The truck staging area may remain adjacent to the residuals handling area, while the chemicals will not have to be pumped as great of a distance to the relevant unit processes.

![New layout](https://drive.google.com/uc?id=10A-rEgVnCaZrJXfaYHl1WWlCH9ITF0JQ)

